In [1]:
from typing import TypedDict
import requests

class CSEAuth(TypedDict):
    cse_id: str
    api_key: str

def validate_cse_auth(
    auth: CSEAuth,
) -> bool:
    """
    Validate the CSEAuth dictionary.

    Args:
        auth (CSEAuth): The CSEAuth dictionary to validate.

    Returns:
        bool: True if valid, False otherwise.
    """
    return all(key in auth for key in ["cse_id", "api_key"])


def validate_image(
    image_url: str,
    ) -> bool:
    
    if not image_url or image_url == "" or not isinstance(image_url, str):
        print(f"Invalid image URL. Expected a non-empty string, found: '{image_url}'")
        return False

    try:
        response = requests.head(image_url)
        return response.status_code >= 200 and response.status_code < 400
    except Exception as e:
        print(f"Error checking image URI: {e}")
        return False

In [6]:
import os
from googleapiclient.discovery import build
from typing import Any, Dict


def custom_search(
    cse_auth: CSEAuth,
    image_url: str,
) -> Dict[str, Any]:
    
    if not validate_cse_auth(cse_auth):
        print("Invalid CSEAuth dictionary.")
        return {}
    if not validate_image(image_url):
        print("Invalid image URL.")
        return {}
    
    service = build("customsearch", "v1", developerKey=cse_auth["api_key"])

    return service.cse().list(
        q=image_url,
        cx=cse_auth["cse_id"],
        searchType="image",
        ).execute()

In [3]:
import sys
import os

# Add the absolute path to the project root
project_root = os.path.abspath("..")
sys.path.append(project_root)


In [4]:
import configparser

with open("../secrets/cse_auth.ini", "r") as f:
    content = f.read()

config = configparser.ConfigParser()
config.read_string(content)

cse_auth: CSEAuth = {
    "cse_id": config["CSE_AUTH"]["CSE_ID"],
    "api_key": config["CSE_AUTH"]["CSE_API_KEY"],
    }

image_url = "https://assets.philosophie.ch/2023-08-10-erni.jpeg"

In [7]:

res = custom_search(
    cse_auth=cse_auth,
    image_url=image_url,
)

In [13]:
res

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - image search',
    'totalResults': '731920000000',
    'searchTerms': 'image search',
    'count': 10,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outputEncoding': 'utf8',
    'safe': 'o

In [ ]:
def reverse_image_search(image_url):
    api_key = "your_serpapi_key"
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": api_key
    }
    response = requests.get("https://serpapi.com/search", params=params)
    return response.json()


In [10]:
def search_similar_images(cse_auth: CSEAuth, image_url):

    search_url = 'https://www.googleapis.com/customsearch/v1'

    params = {
        'q': 'image search',
        'searchType': 'image',
        'imgUrl': image_url,
        'key': cse_auth['api_key'],
        'cx': cse_auth['cse_id']
    }
    response = requests.get(search_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return response.json()

In [11]:
res = search_similar_images(
    cse_auth=cse_auth,
    image_url=image_url,
)

In [14]:
res

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - image search',
    'totalResults': '731920000000',
    'searchTerms': 'image search',
    'count': 10,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outputEncoding': 'utf8',
    'safe': 'o

In [15]:
def reverse_image_search(image_url):
    api_key = "your_serpapi_key"
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": api_key
    }
    response = requests.get("https://serpapi.com/search", params=params)
    return response.json()


In [17]:
import cv2
import pytesseract

def detect_watermark(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    
    if "copyright" in text.lower() or "©" in text:
        return f"Watermark detected in {image_path}"
    return "No watermark found"


In [20]:
detect_watermark("data-copyrighted-imgs/2023-10-15-luporini.jpg")

'No watermark found'